# 01_equipment_for_a_space

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [1]:
import rdflib
import pandas as pd

In [5]:
from _helpers import g

In [37]:
g.parse("01_equipment_for_a_space_model1.ttl", format="turtle");

![](./01_equipment_for_a_space_model.png)

## Query

Basic query that gets all equipment feeding the target space, its children, or its parents.

In [41]:
sparql_query = """
SELECT *
WHERE {
    OPTIONAL {
        {
            BIND("self" as ?rel)
            ?target_room brick:isFedBy ?feed_equip .
            ?feed_equip rdf:type/rdfs:subClassOf* brick:Equipment .
        }
        UNION {
            BIND("parent" as ?rel)
            ?target_room brick:isPartOf+ ?related_room .
            ?related_room rdf:type/rdfs:subClassOf* brick:Location .
            ?related_room brick:isFedBy ?feed_equip .
            ?feed_equip rdf:type/rdfs:subClassOf* brick:Equipment .
        } 
        UNION {
            BIND("child" as ?rel)
            ?target_room brick:hasPart+ ?related_room .
            ?related_room rdf:type/rdfs:subClassOf* brick:Location .
            ?related_room brick:isFedBy ?feed_equip .
            ?feed_equip rdf:type/rdfs:subClassOf* brick:Equipment .
        }
    }
}
"""

In [42]:
res = g.query(sparql_query, initBindings={'target_room': rdflib.Namespace('urn:ex#')['open_plan_working_space']})

In [43]:
res_df = pd.DataFrame(res, columns=[v.toPython() for v in res.vars])
res_df

,?rel,?related_room,?feed_equip,?target_room
0,self,None,urn:ex#fcu_1,urn:ex#open_plan_working_space
1,parent,urn:ex#floor_1,urn:ex#vav_3,urn:ex#open_plan_working_space
2,parent,urn:ex#floor_1,urn:ex#vav_4,urn:ex#open_plan_working_space
3,parent,urn:ex#cubicle_working_space,urn:ex#vav_1,urn:ex#open_plan_working_space
4,parent,urn:ex#cubicle_working_space,urn:ex#vav_2,urn:ex#open_plan_working_space
5,child,urn:ex#hot_desk_space,urn:ex#fcu_2,urn:ex#open_plan_working_space


## Query 2

Same query, but lets add path length

In [133]:
sparql_query = """
SELECT *
WHERE
{
    SELECT ?target_room ?related_room ?rel (count(*) as ?length) ?feed_equip
    WHERE {
        OPTIONAL {
            {
                BIND("self" as ?rel)
                ?target_room brick:isFedBy ?feed_equip .
                ?feed_equip rdf:type/rdfs:subClassOf* brick:Equipment .
            }
            UNION {
                BIND("parent" as ?rel)
                # Get path components between target and related room
                ?target_room brick:isPartOf* ?mid_room .
                ?mid_room brick:isPartOf ?mid_2_room .
                ?mid_2_room brick:isPartOf* ?related_room .
                ?related_room rdf:type/rdfs:subClassOf* brick:Location .
                ?related_room brick:isFedBy ?feed_equip .
                ?feed_equip rdf:type/rdfs:subClassOf* brick:Equipment .
            } 
            UNION {
                BIND("child" as ?rel)
                # Get path components between target and related room
                ?target_room brick:hasPart* ?mid_room .
                ?mid_room brick:hasPart ?mid_2_room .
                ?mid_2_room brick:hasPart* ?related_room .
                ?related_room rdf:type/rdfs:subClassOf* brick:Location .
                ?related_room brick:isFedBy ?feed_equip .
                ?feed_equip rdf:type/rdfs:subClassOf* brick:Equipment .
            }
        }
    }
    GROUP BY ?target_room ?related_room ?feed_equip
}
ORDER BY ?rel ?length
"""

In [134]:
res = g.query(sparql_query, initBindings={'target_room': rdflib.Namespace('urn:ex#')['open_plan_working_space']})

In [135]:
res_df = pd.DataFrame(res, columns=[v.toPython() for v in res.vars])
res_df

,?target_room,?rel,?feed_equip,?length,?related_room
0,urn:ex#open_plan_working_space,child,urn:ex#fcu_2,1,urn:ex#hot_desk_space
1,urn:ex#open_plan_working_space,parent,urn:ex#vav_1,1,urn:ex#cubicle_working_space
2,urn:ex#open_plan_working_space,parent,urn:ex#vav_2,1,urn:ex#cubicle_working_space
3,urn:ex#open_plan_working_space,parent,urn:ex#vav_3,2,urn:ex#floor_1
4,urn:ex#open_plan_working_space,parent,urn:ex#vav_4,2,urn:ex#floor_1
5,urn:ex#open_plan_working_space,self,urn:ex#fcu_1,1,None


Lets try from the perspective of the hot desking space

In [136]:
res = g.query(sparql_query, initBindings={'target_room': rdflib.Namespace('urn:ex#')['hot_desk_space']})

In [137]:
res_df = pd.DataFrame(res, columns=[v.toPython() for v in res.vars])
res_df

,?target_room,?rel,?feed_equip,?length,?related_room
0,urn:ex#hot_desk_space,parent,urn:ex#fcu_1,1,urn:ex#open_plan_working_space
1,urn:ex#hot_desk_space,parent,urn:ex#vav_1,2,urn:ex#cubicle_working_space
2,urn:ex#hot_desk_space,parent,urn:ex#vav_2,2,urn:ex#cubicle_working_space
3,urn:ex#hot_desk_space,parent,urn:ex#vav_3,3,urn:ex#floor_1
4,urn:ex#hot_desk_space,parent,urn:ex#vav_4,3,urn:ex#floor_1
5,urn:ex#hot_desk_space,self,urn:ex#fcu_2,1,None
